# RESTCONF基础

In [1]:
import requests
requests.packages.urllib3.disable_warnings()

In [2]:
login_info = {"host":"192.168.1.61",
              "username":"restconf",
              "password":"restconf"}

# 建立OSPF查看函数

In [3]:
def check_ospf_config():
    ospf_api = "/restconf/data/Cisco-IOS-XE-native:native/router/router-ospf"
    ospf_uri = "https://" + login_info["host"] + ospf_api
    headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

    ospf_check_result = requests.request("GET", 
                                         url=ospf_uri, 
                                         auth=(login_info["username"],login_info["password"]), 
                                         headers=headers, 
                                         verify=False)
    print(ospf_check_result.text)
    
check_ospf_config()

{
  "Cisco-IOS-XE-ospf:router-ospf": {
    "ospf": {
      "process-id": [
        {
          "id": 30,
          "network": [
            {
              "ip": "1.1.1.0",
              "wildcard": "0.0.0.255",
              "area": 0
            }
          ]
        }
      ]
    }
  }
}



# 删除OSPF配置

In [4]:
def del_ospf():
    ospf_api = "/restconf/data/Cisco-IOS-XE-native:native/router/router-ospf"
    ospf_uri = "https://" + login_info["host"] + ospf_api
    headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

    ospf_del_result = requests.request("DELETE",
                                       url=ospf_uri, 
                                       auth=(login_info["username"],login_info["password"]),  
                                       headers=headers, 
                                       verify=False)
    if ospf_del_result.ok:
        print("OSPF配置已清空")
    else:
        print("删除失败，目前仍保留的OSPF配置如下：")
        check_ospf_config()
    
del_ospf()

OSPF配置已清空


In [5]:
check_ospf_config()

# 新增OSPF配置

In [6]:
ospf_data = '''
{
    "Cisco-IOS-XE-native:router": {
        "Cisco-IOS-XE-ospf:router-ospf": {
            "ospf": {
                "process-id": [
                    {
                        "id": 10
                    }
                ]
            }
        }
    }
}
'''

def modify_ospf(data):
    ospf_api = "/restconf/data/Cisco-IOS-XE-native:native/router"
    ospf_uri = "https://" + login_info["host"] + ospf_api
    headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

    ospf_config_result = requests.request("PUT",
                                         url=ospf_uri, 
                                         auth=(login_info["username"],login_info["password"]), 
                                         data=data, 
                                         headers=headers, 
                                         verify=False)
    
    if ospf_config_result.ok:
        print("配置完成，配置数据如下：")
        check_ospf_config()
    else:
        print("配置失败，原因如下：")
        print(ospf_config_result.reason)
    
modify_ospf(ospf_data)

配置完成，配置数据如下：
{
  "Cisco-IOS-XE-ospf:router-ospf": {
    "ospf": {
      "process-id": [
        {
          "id": 10
        }
      ]
    }
  }
}



# 新增环回口 100

In [7]:
loop_data = '''
{
    "Cisco-IOS-XE-native:Loopback": [
        {
            "name": 100,
            "ip": {
                "address": {
                    "primary": {
                        "address": "100.100.100.100",
                        "mask": "255.255.255.0"
                    }
                }
            }
        }
    ]
}
'''
def loop_add(data):
    loop_api = "/restconf/data/Cisco-IOS-XE-native:native/interface"
    loop_uri = "https://" + login_info["host"] + loop_api
    headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

    loop_add_result = requests.request("POST",
                                       url=loop_uri, 
                                       auth=(login_info["username"],login_info["password"]), 
                                       data=data, 
                                       headers=headers, 
                                       verify=False)
    
    if loop_add_result.ok:
        print("配置完成，配置数据如下：")
        check_loop()
    else:
        print("配置失败，原因如下：")
        print(loop_add_result.reason)
        
def check_loop():
    loop_api = "/restconf/data/Cisco-IOS-XE-native:native/interface/Loopback"
    loop_uri = "https://" + login_info["host"] + loop_api
    headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

    loop_check_result = requests.request("GET",
                                         url=loop_uri, 
                                         auth=(login_info["username"],login_info["password"]), 
                                         headers=headers, 
                                         verify=False)
    print(loop_check_result.text)
    
loop_add(loop_data)

配置完成，配置数据如下：
{
  "Cisco-IOS-XE-native:Loopback": [
    {
      "name": 0,
      "ip": {
        "address": {
          "primary": {
            "address": "1.1.1.1",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 1,
      "ip": {
        "address": {
          "primary": {
            "address": "2.2.2.2",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 3,
      "ip": {
        "address": {
          "primary": {
            "address": "3.3.3.3",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 4,
      "ip": {
        "address": {
          "primary": {
            "address": "4.4.4.4",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 5,
      "ip": {
        "address": {
          "primary": {
            "address": "5.5.5.5",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
   

# 将Loopback100发布到OSPF进程

In [8]:
loop_ospf_data = '''
{
    "Cisco-IOS-XE-native:Loopback": [
        {
            "name": 100,
            "ip": {
                "address": {
                    "primary": {
                        "address": "100.100.100.100",
                        "mask": "255.255.255.0"
                    }
                },
                "Cisco-IOS-XE-ospf:router-ospf": {
                    "ospf": {
                        "process-id": [
                            {
                                "id": 10,
                                "area": [
                                    {
                                        "area-id": 0
                                    }
                                ]
                            }
                        ]
                    }
                }
            }
        }
    ]
}
'''
def loop_ospf(data):
    loop_api = "/restconf/data/Cisco-IOS-XE-native:native/interface/Loopback=100"
    loop_uri = "https://" + login_info["host"] + loop_api
    headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

    loop_ospf_result = requests.request("PUT",
                                        url=loop_uri, 
                                        auth=(login_info["username"],login_info["password"]), 
                                        data=data, 
                                        headers=headers, 
                                        verify=False)
    
    if loop_ospf_result.ok:
        print("配置完成，配置数据如下：")
        check_loop()
    else:
        print("配置失败，原因如下：")
        print(loop_ospf_result.reason)
    
loop_ospf(loop_ospf_data)

配置完成，配置数据如下：
{
  "Cisco-IOS-XE-native:Loopback": [
    {
      "name": 0,
      "ip": {
        "address": {
          "primary": {
            "address": "1.1.1.1",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 1,
      "ip": {
        "address": {
          "primary": {
            "address": "2.2.2.2",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 3,
      "ip": {
        "address": {
          "primary": {
            "address": "3.3.3.3",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 4,
      "ip": {
        "address": {
          "primary": {
            "address": "4.4.4.4",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 5,
      "ip": {
        "address": {
          "primary": {
            "address": "5.5.5.5",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
   

In [9]:
check_ospf_config()

{
  "Cisco-IOS-XE-ospf:router-ospf": {
    "ospf": {
      "process-id": [
        {
          "id": 10
        }
      ]
    }
  }
}



# 删除环回接口

In [10]:
def loop_del():
    loop_api = "/restconf/data/Cisco-IOS-XE-native:native/interface/Loopback=100"
    loop_uri = "https://" + login_info["host"] + loop_api
    headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

    loop_del_result = requests.request("DELETE",
                                        url=loop_uri, 
                                        auth=(login_info["username"],login_info["password"]), 
                                        headers=headers, 
                                        verify=False)
    
    if loop_del_result.ok:
        print("环回接口已删除，目前环回接口配置如下：")
        check_loop()
    else:
        print("删除失败，目前环回接口配置如下：")
        check_loop()
    
loop_del()

环回接口已删除，目前环回接口配置如下：
{
  "Cisco-IOS-XE-native:Loopback": [
    {
      "name": 0,
      "ip": {
        "address": {
          "primary": {
            "address": "1.1.1.1",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 1,
      "ip": {
        "address": {
          "primary": {
            "address": "2.2.2.2",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 3,
      "ip": {
        "address": {
          "primary": {
            "address": "3.3.3.3",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 4,
      "ip": {
        "address": {
          "primary": {
            "address": "4.4.4.4",
            "mask": "255.255.255.0"
          }
        }
      }
    },
    {
      "name": 5,
      "ip": {
        "address": {
          "primary": {
            "address": "5.5.5.5",
            "mask": "255.255.255.0"
          }
        }
      }
    }
  ]

# 删除OSPF配置

In [11]:
del_ospf()

OSPF配置已清空


In [12]:
check_ospf_config()